In [27]:
# Load Libraries
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from metpy.units import units
import numpy as np
import matplotlib as mpl
from collections import OrderedDict
import glob
import os

In [28]:
# Function to calculate the mean specific humidity between two pressure levels
def calc_mean_layer_p(alt_arr,pres_lvls,data):

    # Get pressure level indicies
    alt0 = np.where(np.abs(np.array(pres_lvls) - alt_arr[0]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(pres_lvls) - alt_arr[1]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[1])))

    # Get specific data and do calculation
    var1 = data[int(alt0[0]):int(alt1[0])]
    var = np.nanmean(var1)
    del(var1)
    
    return var

def calc_mean_layer_z(alt_arr,pres_lvls,data):

    # Get pressure level indicies
    alt0 = np.where(np.abs(np.array(pres_lvls) - alt_arr[0]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(pres_lvls) - alt_arr[1]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[1])))

    # Get specific data and do calculation
    var1 = data[int(alt0[0]):int(alt1[0])]
    var = np.nanmean(var1)
    del(var1)
    
    return var


# Function to calculate the wind shear between two pressure levels
def calc_layer_shear_p(alt_arr,pres_lvls,udata,vdata):

    #### LOW LEVEL SHEAR        
#    alt_arr = [800, 1000]
    alt0 = np.where(np.abs(np.array(pres_lvls) - alt_arr[0]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(pres_lvls) - alt_arr[1]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[1])))

    var1 = udata[int(alt0[0])]
    var2 = vdata[int(alt0[0])]
    var3 = udata[int(alt1[0])]
    var4 = vdata[int(alt1[0])]
    var = np.sqrt(np.power((var3-var1),2.0) + np.power((var4-var2),2.0))
    
    return var

def calc_layer_shear_z(alt_arr,alt_lvls,udata,vdata):

    #### LOW LEVEL SHEAR        
#    alt_arr = [800, 1000]
    alt0 = np.where(np.abs(np.array(alt_lvls) - alt_arr[0]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(alt_lvls) - alt_arr[1]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[1])))

    var1 = udata[int(alt0[0])]
    var2 = vdata[int(alt0[0])]
    var3 = udata[int(alt1[0])]
    var4 = vdata[int(alt1[0])]
    var = np.sqrt(np.power((var3-var1),2.0) + np.power((var4-var2),2.0))
    
    return var


def calc_tcwv(pres,temp,rv,zmn,topt):
    rd = 287; #J/kg/K
    dens = pres/(rd*temp*(1+0.61*rv)) # calculate dense
    del(pres,temp)
    
    # Difference in heights (dz)    
    diff_zt = np.diff(zmn)*(1-topt/np.nanmax(zmn))
    
    # integrated total column water vapor in kg / m^2
    tcwv = np.nansum(rv[1:]*dens[1:]*diff_zt)
    del(rv,diff_zt,dens)

    return tcwv    

In [29]:
def read_head(headfile,h5file):
    
    # Function that reads header files from RAMS
    
    # Inputs:
    #   headfile: header file including full path in str format
    #   h5file: h5 datafile including full path in str format
    
    # Returns:
    #   zmn: height levels for momentum values (i.e., grid box upper and lower levels)
    #   ztn: height levels for thermodynaic values (i.e., grid box centers)
    #   nx:: the number of x points for the domain associated with the h5file
    #   ny: the number of y points for the domain associated with the h5file
    #   npa: the number of surface patches
    
    
    dom_num = h5file[h5file.index('.h5')-1] # Find index of .h5 to determine position showing which nest domain to use

    with open(headfile) as f:
        contents = f.readlines()
        
    idx_zmn = contents.index('__zmn0'+dom_num+'\n')
    nz_m = int(contents[idx_zmn+1])
    zmn = np.zeros(nz_m)
    for i in np.arange(0,nz_m):
        zmn[i] =  float(contents[idx_zmn+2+i])
    
    idx_ztn = contents.index('__ztn0'+dom_num+'\n')
    nz_t = int(contents[idx_ztn+1])
    ztn = np.zeros(nz_t)
    for i in np.arange(0,nz_t):
        ztn[i] =  float(contents[idx_ztn+2+i])
    
    ztop = np.max(ztn) # Model domain top (m)
    
    # Grad the size of the horizontal grid spacing
    idx_dxy = contents.index('__deltaxn\n')
    dxy = float(contents[idx_dxy+1+int(dom_num)].strip())

    idx_npatch = contents.index('__npatch\n')
    npa = int(contents[idx_npatch+2])
    
    idx_ny = contents.index('__nnyp\n')
    idx_nx = contents.index('__nnxp\n')
    ny = np.ones(int(contents[idx_ny+1]))
    nx = np.ones(int(contents[idx_ny+1]))
    for i in np.arange(0,len(ny)):
        nx[i] = int(contents[idx_nx+2+i])
        ny[i] = int(contents[idx_ny+2+i])

    ny_out = ny[int(dom_num)-1]
    nx_out = nx[int(dom_num)-1]

    return zmn, ztn, nx_out, ny_out, dxy, npa 

In [30]:
mpl.rcParams.update({'font.size': 15})
lwid = 3.0

In [31]:
#cases = ['ARG1.1-R_old','ARG1.2-R','BRA1.1-R']
cases = ['ARG1.1-R_old','ARG1.2-R','BRA1.1-R','BRA1.2-R','AUS1.1-R','DRC1.1-R','PHI1.1-R','PHI1.1-RPR','PHI2.1-R','WPO1.1-R','USA1.1-R','RSA1.1-R']

In [32]:

shr_ml = OrderedDict()
shr_ll = OrderedDict()

rh_ml = OrderedDict()
rh_ll = OrderedDict()
rh_sfc = OrderedDict()
rh_850 = OrderedDict()
rh_500 = OrderedDict()
rh_250 = OrderedDict()

rv_ml = OrderedDict()
rv_ll = OrderedDict()
rv_sfc = OrderedDict()
rv_850 = OrderedDict()
rv_500 = OrderedDict()
rv_250 = OrderedDict()

spd_ml = OrderedDict()
spd_ll = OrderedDict()
spd_sfc = OrderedDict()
spd_850 = OrderedDict()
spd_500 = OrderedDict()
spd_250 = OrderedDict()

t_sfc = OrderedDict()
t_ll = OrderedDict()
t_ml = OrderedDict()
t_850 = OrderedDict()
t_500 = OrderedDict()
t_250 = OrderedDict()

fl_p = OrderedDict()
lcl_p = OrderedDict()
lcl_z = OrderedDict()
lcl_t = OrderedDict()
wcd_p = OrderedDict()

mucape = OrderedDict()
mucin = OrderedDict()
mlcape = OrderedDict()
mlcin = OrderedDict()

tcw = OrderedDict()
tcwv = OrderedDict()
tcwc = OrderedDict()


shr_mls = OrderedDict()
shr_lls = OrderedDict()

rh_mls = OrderedDict()
rh_lls = OrderedDict()
rh_sfcs = OrderedDict()
rh_850s = OrderedDict()
rh_500s = OrderedDict()
rh_250s = OrderedDict()

rv_mls = OrderedDict()
rv_lls = OrderedDict()
rv_sfcs = OrderedDict()
rv_850s = OrderedDict()
rv_500s = OrderedDict()
rv_250s = OrderedDict()

spd_mls = OrderedDict()
spd_lls = OrderedDict()
spd_sfcs = OrderedDict()
spd_850s = OrderedDict()
spd_500s = OrderedDict()
spd_250s = OrderedDict()

t_sfcs = OrderedDict()
t_lls = OrderedDict()
t_mls = OrderedDict()
t_850s = OrderedDict()
t_500s = OrderedDict()
t_250s = OrderedDict()

fl_ps = OrderedDict()
lcl_ps = OrderedDict()
lcl_zs = OrderedDict()
lcl_ts = OrderedDict()
wcd_ps = OrderedDict()

mucapes = OrderedDict()
mucins = OrderedDict()
mlcapes = OrderedDict()
mlcins = OrderedDict()

tcws = OrderedDict()
tcwvs = OrderedDict()
tcwcs = OrderedDict()

#spacings = [3,4,5,7,9] # 9km, 5km, 3km, 1km 
spacings = [3,4,5,9,27] # 9km, 7km, 5km, 3km, 1km 
pmllvls = [800,400]
plllvls = [1000,800]
#zmllvls = [0,2000]
#zlllvls = [3000,8000]
for c in np.arange(0,len(cases)):
    cn = cases[c]
    print(cn)
    files = sorted(glob.glob('/monsoon/LES_MODEL_DATA/'+cn+'/G3/out_30s/a-A*g3.h5'))
    hfiles = sorted(glob.glob('/monsoon/LES_MODEL_DATA/'+cn+'/G3/out_30s/a-A*head.txt'))
    zmn, ztn, nx_out, ny_out, dxy, npa = read_head(hfiles[0],files[0])

    infilename = '/monsoon/pmarin/ENV/Sim_Profs_range_all_'+cn+'.p'
    savepath = '/monsoon/pmarin/ENV/'+cn+'/'
    plotpath = '/monsoon/pmarin/ENV/Plots/'+cn+'/'
    if not os.path.exists(plotpath):
        os.makedirs(plotpath)

    if not os.path.exists(savepath):
        os.makedirs(savepath)

    with open(infilename, 'rb') as f:
        [profd,latlon,numg,profds,latlons,latlons_id,numgs] = pickle.load(f)    
        num_prof = numg[cn]
        print(num_prof)

    shr_ml[cn] = np.zeros(num_prof)
    shr_ll[cn] = np.zeros(num_prof)

    rh_sfc[cn] = np.zeros(num_prof)
    rh_ml[cn] = np.zeros(num_prof)
    rh_ll[cn] = np.zeros(num_prof)
    rh_850[cn] = np.zeros(num_prof)
    rh_500[cn] = np.zeros(num_prof)
    rh_250[cn] = np.zeros(num_prof)

    rv_sfc[cn] = np.zeros(num_prof)
    rv_ml[cn] = np.zeros(num_prof)
    rv_ll[cn] = np.zeros(num_prof)
    rv_850[cn] = np.zeros(num_prof)
    rv_500[cn] = np.zeros(num_prof)
    rv_250[cn] = np.zeros(num_prof)
    
    spd_sfc[cn] = np.zeros(num_prof)
    spd_ml[cn] = np.zeros(num_prof)
    spd_ll[cn] = np.zeros(num_prof)
    spd_850[cn] = np.zeros(num_prof)
    spd_500[cn] = np.zeros(num_prof)
    spd_250[cn] = np.zeros(num_prof)

    t_sfc[cn] = np.zeros(num_prof)
    t_ml[cn] = np.zeros(num_prof)
    t_ll[cn] = np.zeros(num_prof)
    t_850[cn] = np.zeros(num_prof)
    t_500[cn] = np.zeros(num_prof)
    t_250[cn] = np.zeros(num_prof)

    fl_p[cn] = np.zeros(num_prof)
    lcl_t[cn] = np.zeros(num_prof)
    lcl_p[cn] = np.zeros(num_prof)
    lcl_z[cn] = np.zeros(num_prof)
    wcd_p[cn] = np.zeros(num_prof)
    mucape[cn] = np.zeros(num_prof)
    mucin[cn] = np.zeros(num_prof)
    mlcape[cn] = np.zeros(num_prof)
    mlcin[cn] = np.zeros(num_prof)
    tcwv[cn] = np.zeros(num_prof)
    tcwc[cn] = np.zeros(num_prof)
    tcw[cn] = np.zeros(num_prof)
    
    for i in np.arange(0,num_prof):
        print(i)
#        if i == 15 and cn == 'PHI1.1-R':
#            continue
            
        temp = profd[cn]['temp'][i]
        pres = profd[cn]['pres'][i]/100
        rv = profd[cn]['RV'][i]
        u = profd[cn]['UP'][i]
        v = profd[cn]['VP'][i]       
        rtc = profd[cn]['RTC'][i]
        topt = profd[cn]['topt'][i]
        spd = np.sqrt(u*u+v*v)
        
        #print(pres[0])
        idsfc = 1
        id850 = np.where(np.abs(pres-850) == np.min(np.abs(pres-850)))[0][0]
        id500 = np.where(np.abs(pres-500) == np.min(np.abs(pres-500)))[0][0]
        id250 = np.where(np.abs(pres-250) == np.min(np.abs(pres-250)))[0][0]

        t_sfc[cn][i] = temp[1]
        rv_sfc[cn][i] = rv[1]
        spd_sfc[cn][i] = spd[1]
        
        t_850[cn][i] = temp[id850]
        rv_850[cn][i] = rv[id850]
        spd_850[cn][i] = spd[id850]

        t_500[cn][i] = temp[id500]
        rv_500[cn][i] = rv[id500]
        spd_500[cn][i] = spd[id500]

        t_250[cn][i] = temp[id250]
        rv_250[cn][i] = rv[id250]
        spd_250[cn][i] = spd[id250]

        t_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,temp)
        rv_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,rv)
        spd_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,spd)

        t_ll[cn][i] = calc_mean_layer_p(plllvls,pres,temp)
        rv_ll[cn][i] = calc_mean_layer_p(plllvls,pres,rv)
        spd_ll[cn][i] = calc_mean_layer_p(plllvls,pres,spd)                
                
        tcwv[cn][i] = calc_tcwv(pres*100,temp,rv,zmn,topt)
        tcwc[cn][i] = calc_tcwv(pres*100,temp,rtc,zmn,topt)
        tcw[cn][i] = calc_tcwv(pres*100,temp,rtc+rv,zmn,topt)

        id_0c = np.where(np.abs(temp-273.15) == np.min(np.abs((temp-273.15))))[0]
        fl_p[cn][i] = pres[int(id_0c[0])]
        
        # Save temperature and shear data
        shr_ml[cn][i] = calc_layer_shear_p(pmllvls,pres,u,v)
        shr_ll[cn][i] = calc_layer_shear_p(plllvls,pres,u,v)        
        
        # convert profile to variables / units for metpy
        p_mp = pres*units.hPa

        t_mp = temp-273.15
        t_mp = t_mp*units.degC

        # Calculate vapor pressure
        mixing = rv*units('kg/kg')
        e_mp = mpcalc.vapor_pressure(p_mp, mixing)
        e_mp[e_mp==0]=np.NaN
        e_mp[np.isnan(e_mp)]=np.nanmin(e_mp)

        es_mp = mpcalc.saturation_vapor_pressure(t_mp).to('hPa')
        rh = (e_mp/es_mp).magnitude
#        print(rh)

        # Save rh data
        rh_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,rh)
        rh_ll[cn][i] = calc_mean_layer_p(plllvls,pres,rh)
        rh_sfc[cn][i] = rh[1]     
        rh_850[cn][i] = rh[id850]     
        rh_500[cn][i] = rh[id500]     
        rh_250[cn][i] = rh[id250]     
        
        # Caclculate depoint temperature
        td_mp = mpcalc.dewpoint(e_mp)

        # Convert winds from m/s to knots
        u_mp = u*1.94384*units.knots
        v_mp = v*1.94384*units.knots

        fig = plt.figure(figsize=(9, 9))
        ##add_metpy_logo(fig, 115, 100)
        skew = SkewT(fig, rotation=45)

        # Plot the data using normal plotting functions, in this case using
        # log scaling in Y, as dictated by the typical meteorological plot.
        skew.plot(p_mp, t_mp, 'r',linewidth=lwid)
        skew.plot(p_mp, td_mp, 'g',linewidth=lwid)
        ##skew.plot_barbs(p, u, v)
        skew.ax.set_ylim(1000, 100)
        skew.ax.set_xlim(-30, 40)

        # Calculate LCL height and plot as black dot. Because `p`'s first value is
        # ~1000 mb and its last value is ~250 mb, the `0` index is selected for
        # `p`, `T`, and `Td` to lift the parcel from the surface. If `p` was inverted,
        # i.e. start from low value, 250 mb, to a high value, 1000 mb, the `-1` index
        # should be selected.
        lcl_pressure, lcl_temperature = mpcalc.lcl(p_mp[0], t_mp[0], td_mp[0])
        lcl_p[cn][i] = lcl_pressure.magnitude
        lcl_t[cn][i] = lcl_temperature.magnitude
        skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black',linewidth=lwid)

        wcd_p[cn][i] = lcl_p[cn][i]-fl_p[cn][i]
              
        # Calculate full parcel profile and add to plot as black line
        prof = mpcalc.parcel_profile(p_mp, t_mp[0], td_mp[0]).to('degC')
        skew.plot(p_mp, prof, 'k',linewidth=lwid)
        cape, cin = mpcalc.cape_cin(p_mp, t_mp, td_mp, prof)
        p_mlcape, p_mlcin = mpcalc.mixed_layer_cape_cin(p_mp, t_mp, td_mp)
        p_mucape, p_mucin = mpcalc.most_unstable_cape_cin(p_mp, t_mp, td_mp)

        mlcape[cn][i] = p_mlcape.magnitude
        mlcin[cn][i] = p_mlcin.magnitude
        mucape[cn][i] = p_mucape.magnitude
        mucin[cn][i] = p_mucin.magnitude
        
        # Shade areas of CAPE and CIN
        #skew.shade_cin(p_mp, t_mp, prof, td_mp)
        #skew.shade_cape(p_mp, t_mp, prof)

        # An example of a slanted line at constant T -- in this case the 0
        # isotherm
        skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

        # Add the relevant special lines
        skew.plot_dry_adiabats()
        skew.plot_moist_adiabats()
        skew.plot_mixing_lines()

        plt.xlabel('Temperature (C)')
        plt.ylabel('Pressure (hPa)')

        plt.title(cn+': Dom'+str(i)+' | CAPE:'+str(int(p_mlcape.magnitude))+' CIN:'+str(int(p_mlcin.magnitude)))
        # Show the plot
#        plt.show()   
        plt.tight_layout()
        fig.savefig(plotpath+'SkewT_LogP_'+cn+str(i)+'_MLCAPE.png')    
        plt.close(fig)

    save_vars = OrderedDict()
    save_vars['shr_mls'] = shr_ml
    save_vars['shr_lls'] = shr_ll

    save_vars['rh_mls'] = rh_ml
    save_vars['rh_lls'] = rh_ll
    save_vars['rh_sfcs'] = rh_sfc
    save_vars['rh_850s'] = rh_850
    save_vars['rh_500s'] = rh_500
    save_vars['rh_250s'] = rh_250

    save_vars['rv_mls'] = rv_ml
    save_vars['rv_lls'] = rv_ll
    save_vars['rv_sfcs'] = rv_sfc
    save_vars['rv_850s'] = rv_850
    save_vars['rv_500s'] = rv_500
    save_vars['rv_250s'] = rv_250

    save_vars['t_mls'] = t_ml
    save_vars['t_lls'] = t_ll
    save_vars['t_sfcs'] = t_sfc
    save_vars['t_850s'] = t_850
    save_vars['t_500s'] = t_500
    save_vars['t_250s'] = t_250                

    save_vars['spd_mls'] = spd_ml
    save_vars['spd_lls'] = spd_ll
    save_vars['spd_sfcs'] = spd_sfc
    save_vars['spd_850s'] = spd_850
    save_vars['spd_500s'] = spd_500
    save_vars['spd_250s'] = spd_250                                
                
    save_vars['rh_mls'] = rh_ml
    save_vars['rh_lls'] = rh_ll
    save_vars['rh_sfcs'] = rh_sfc
    save_vars['rh_850s'] = rh_850
    save_vars['rh_500s'] = rh_500
    save_vars['rh_250s'] = rh_250
                
    save_vars['fl_ps'] = fl_p
    save_vars['lcl_ts'] = lcl_t
    save_vars['lcl_ps'] = lcl_p
    save_vars['lcl_zs'] = lcl_z
    save_vars['wcd_ps'] = wcd_p
    save_vars['t_sfcs'] = t_sfc
    save_vars['mucapes'] = mucape
    save_vars['mucins'] = mucin
    save_vars['mlcapes'] = mlcape
    save_vars['mlcins'] = mlcin
    save_vars['tcwvs'] = tcwv
    save_vars['tcwcs'] = tcwc
    save_vars['tcws'] = tcw
    savefilename = cn+'_Environments_ERA5.p'
    with open(savepath+savefilename, "wb") as output_file:
        pickle.dump(save_vars, output_file)

    # Start calculations for subdomains
    subs = numg[cn]
    # Loop through ERA5 subdomains
    for s in np.arange(0,subs):
        print(s)
        # Create filename for each ERA5 subdomain
        savefilename = cn+'_Environments_SubERA5_'+str(s)+'.p'
        for gs in np.arange(0,len(spacings)):

            ss = spacings[gs]

            num_prof = numgs[cn,s,gs] # number of subdomains with era5 subdomain
            print(num_prof)

            shr_mls[cn,gs] = np.zeros(num_prof)
            shr_lls[cn,gs] = np.zeros(num_prof)

            rh_sfcs[cn,gs] = np.zeros(num_prof)
            rh_mls[cn,gs] = np.zeros(num_prof)
            rh_lls[cn,gs] = np.zeros(num_prof)
            rh_850s[cn,gs] = np.zeros(num_prof)
            rh_500s[cn,gs] = np.zeros(num_prof)
            rh_250s[cn,gs] = np.zeros(num_prof)

            rv_sfcs[cn,gs] = np.zeros(num_prof)
            rv_mls[cn,gs] = np.zeros(num_prof)
            rv_lls[cn,gs] = np.zeros(num_prof)
            rv_850s[cn,gs] = np.zeros(num_prof)
            rv_500s[cn,gs] = np.zeros(num_prof)
            rv_250s[cn,gs] = np.zeros(num_prof)                

            spd_sfcs[cn,gs] = np.zeros(num_prof)
            spd_mls[cn,gs] = np.zeros(num_prof)
            spd_lls[cn,gs] = np.zeros(num_prof)
            spd_850s[cn,gs] = np.zeros(num_prof)
            spd_500s[cn,gs] = np.zeros(num_prof)
            spd_250s[cn,gs] = np.zeros(num_prof)                
                
            t_sfcs[cn,gs] = np.zeros(num_prof)
            t_mls[cn,gs] = np.zeros(num_prof)
            t_lls[cn,gs] = np.zeros(num_prof)
            t_850s[cn,gs] = np.zeros(num_prof)
            t_500s[cn,gs] = np.zeros(num_prof)
            t_250s[cn,gs] = np.zeros(num_prof)                

            fl_ps[cn,gs] = np.zeros(num_prof)
            lcl_ts[cn,gs] = np.zeros(num_prof)
            lcl_ps[cn,gs] = np.zeros(num_prof)
            lcl_zs[cn,gs] = np.zeros(num_prof)
            wcd_ps[cn,gs] = np.zeros(num_prof)
            mucapes[cn,gs] = np.zeros(num_prof)
            mucins[cn,gs] = np.zeros(num_prof)
            mlcapes[cn,gs] = np.zeros(num_prof)
            mlcins[cn,gs] = np.zeros(num_prof)
            tcwvs[cn,gs] = np.zeros(num_prof)
            tcwcs[cn,gs] = np.zeros(num_prof)
            tcws[cn,gs] = np.zeros(num_prof)

            for i in np.arange(0,num_prof):

        #        if i == 15 and cn == 'PHI1.1-R':
        #            continue

                temp = profds[cn]['temp'][s][ss][i]
                pres = profds[cn]['pres'][s][ss][i]/100
                rv = profds[cn]['RV'][s][ss][i]
                u = profds[cn]['UP'][s][ss][i]
                v = profds[cn]['VP'][s][ss][i]       
                rtc = profds[cn]['RTC'][s][ss][i]
                topt = profds[cn]['topt'][s][ss][i]
                spd = np.sqrt(u*u+v*v)

                #print(pres[0])
                idsfc = 1
                id850 = np.where(np.abs(pres-850) == np.min(np.abs(pres-850)))[0][0]
                id500 = np.where(np.abs(pres-500) == np.min(np.abs(pres-500)))[0][0]
                id250 = np.where(np.abs(pres-250) == np.min(np.abs(pres-250)))[0][0]

                t_sfcs[cn,gs][i] = temp[1]
                rv_sfcs[cn,gs][i] = rv[1]
                spd_sfcs[cn,gs][i] = spd[1]

                t_850s[cn,gs][i] = temp[id850]
                rv_850s[cn,gs][i] = rv[id850]
                spd_850s[cn,gs][i] = spd[id850]

                t_500s[cn,gs][i] = temp[id500]
                rv_500s[cn,gs][i] = rv[id500]
                spd_500s[cn,gs][i] = spd[id500]

                t_250s[cn,gs][i] = temp[id250]
                rv_250s[cn,gs][i] = rv[id250]
                spd_250s[cn,gs][i] = spd[id250]

                t_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,temp)
                t_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,temp)

                rv_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,rv)
                rv_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,rv)
                
                spd_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,spd)
                spd_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,spd)
                
                tcwvs[cn,gs][i] = calc_tcwv(pres*100,temp,rv,zmn,topt)               
                tcwcs[cn,gs][i] = calc_tcwv(pres*100,temp,rtc,zmn,topt)               
                tcws[cn,gs][i] = calc_tcwv(pres*100,temp,rtc+rv,zmn,topt)               
                
                id_0c = np.where(np.abs(temp-273.15) == np.min(np.abs((temp-273.15))))[0]
                fl_ps[cn,gs][i] = pres[int(id_0c[0])]

                # Save temperature and shear data
                shr_mls[cn,gs][i] = calc_layer_shear_p(pmllvls,pres,u,v)
                shr_lls[cn,gs][i] = calc_layer_shear_p(plllvls,pres,u,v)        

                # convert profile to variables / units for metpy
                p_mp = pres*units.hPa

                t_mp = temp-273.15
                t_mp = t_mp*units.degC

                # Calculate vapor pressure
                mixing = rv*units('kg/kg')
                e_mp = mpcalc.vapor_pressure(p_mp, mixing)
                e_mp[e_mp==0]=np.NaN
                e_mp[np.isnan(e_mp)]=np.nanmin(e_mp)

                es_mp = mpcalc.saturation_vapor_pressure(t_mp).to('hPa')
                rh = (e_mp/es_mp).magnitude
        #        print(rh)

                # Save rh data
                rh_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,rh)
                rh_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,rh)
                rh_sfcs[cn,gs][i] = rh[1]     
                rh_850s[cn,gs][i] = rh[id850]
                rh_500s[cn,gs][i] = rh[id500]
                rh_250s[cn,gs][i] = rh[id250]

                # Caclculate depoint temperature
                td_mp = mpcalc.dewpoint(e_mp)

                # Convert winds from m/s to knots
                u_mp = u*1.94384*units.knots
                v_mp = v*1.94384*units.knots

                #fig = plt.figure(figsize=(9, 9))
                #skew = SkewT(fig, rotation=45)
                                
                # Plot the data using normal plotting functions, in this case using
                # log scaling in Y, as dictated by the typical meteorological plot.
                #skew.plot(p_mp, t_mp, 'r',linewidth=lwid)
                #skew.plot(p_mp, td_mp, 'g',linewidth=lwid)
                ##skew.plot_barbs(p, u, v)
                #skew.ax.set_ylim(1000, 100)
                #skew.ax.set_xlim(-30, 40)

                # Calculate LCL height and plot as black dot. Because `p`'s first value is
                # ~1000 mb and its last value is ~250 mb, the `0` index is selected for
                # `p`, `T`, and `Td` to lift the parcel from the surface. If `p` was inverted,
                # i.e. start from low value, 250 mb, to a high value, 1000 mb, the `-1` index
                # should be selected.
                lcl_pressure, lcl_temperature = mpcalc.lcl(p_mp[0], t_mp[0], td_mp[0])
                lcl_ps[cn,gs][i] = lcl_pressure.magnitude
                lcl_ts[cn,gs][i] = lcl_temperature.magnitude
                #skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black',linewidth=lwid)

                wcd_ps[cn,gs][i] = lcl_ps[cn,gs][i]-fl_ps[cn,gs][i]

                # Calculate full parcel profile and add to plot as black line
                prof = mpcalc.parcel_profile(p_mp, t_mp[0], td_mp[0]).to('degC')
                #skew.plot(p_mp, prof, 'k',linewidth=lwid)
                cape, cin = mpcalc.cape_cin(p_mp, t_mp, td_mp, prof)
                p_mlcape, p_mlcin = mpcalc.mixed_layer_cape_cin(p_mp, t_mp, td_mp)
                p_mucape, p_mucin = mpcalc.most_unstable_cape_cin(p_mp, t_mp, td_mp)

                mlcapes[cn,gs][i] = p_mlcape.magnitude
                mlcins[cn,gs][i] = p_mlcin.magnitude
                mucapes[cn,gs][i] = p_mucape.magnitude
                mucins[cn,gs][i] = p_mucin.magnitude

                # Shade areas of CAPE and CIN
                #skew.shade_cin(p_mp, t_mp, prof, td_mp)
                #skew.shade_cape(p_mp, t_mp, prof)

                # An example of a slanted line at constant T -- in this case the 0
                # isotherm
                #skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

                # Add the relevant special lines
                #skew.plot_dry_adiabats()
                #skew.plot_moist_adiabats()
                #skew.plot_mixing_lines()

                #plt.xlabel('Temperature (C)')
                #plt.ylabel('Pressure (hPa)')

                #plt.title(cn+': Dom'+str(i)+' | CAPE:'+str(int(p_mlcape.magnitude))+' CIN:'+str(int(p_mlcin.magnitude)))
                # Show the plot
        #        plt.show()   
                #plt.tight_layout()
                #fig.savefig(plotpath+'SkewT_LogP_'+cn+str(i)+'_MLCAPE'+str(s)+'_'+str(gs)+'.png')    
                #plt.close(fig)

            save_vars = OrderedDict()
            save_vars['shr_mls'] = shr_mls
            save_vars['shr_lls'] = shr_lls

            save_vars['rh_mls'] = rh_mls
            save_vars['rh_lls'] = rh_lls
            save_vars['rh_sfcs'] = rh_sfcs
            save_vars['rh_850s'] = rh_850s
            save_vars['rh_500s'] = rh_500s
            save_vars['rh_250s'] = rh_250s

            save_vars['rv_mls'] = rv_mls
            save_vars['rv_lls'] = rv_lls
            save_vars['rv_sfcs'] = rv_sfcs
            save_vars['rv_850s'] = rv_850s
            save_vars['rv_500s'] = rv_500s
            save_vars['rv_250s'] = rv_250s

            save_vars['t_mls'] = t_mls
            save_vars['t_lls'] = t_lls
            save_vars['t_sfcs'] = t_sfcs
            save_vars['t_850s'] = t_850s
            save_vars['t_500s'] = t_500s
            save_vars['t_250s'] = t_250s               

            save_vars['spd_mls'] = spd_mls
            save_vars['spd_lls'] = spd_lls
            save_vars['spd_sfcs'] = spd_sfcs
            save_vars['spd_850s'] = spd_850s
            save_vars['spd_500s'] = spd_500s
            save_vars['spd_250s'] = spd_250s                               

            save_vars['rh_mls'] = rh_mls
            save_vars['rh_lls'] = rh_lls
            save_vars['rh_sfcs'] = rh_sfcs
            save_vars['rh_850s'] = rh_850s
            save_vars['rh_500s'] = rh_500s
            save_vars['rh_250s'] = rh_250s               
                
            save_vars['fl_ps'] = fl_ps
            save_vars['lcl_ts'] = lcl_ts
            save_vars['lcl_ps'] = lcl_ps
            save_vars['lcl_zs'] = lcl_zs
            save_vars['wcd_ps'] = wcd_ps
            save_vars['mucapes'] = mucapes
            save_vars['mucins'] = mucins
            save_vars['mlcapes'] = mlcapes
            save_vars['mlcins'] = mlcins
            save_vars['tcwvs'] = tcwvs
            save_vars['tcwcs'] = tcwvs
            save_vars['tcws'] = tcwvs

        with open(savepath+savefilename, "wb") as output_file:
            pickle.dump(save_vars, output_file)
        
        

ARG1.1-R_old
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0
9
16
25
81
729
1
9
16
25
81
729
2
9
16
25
81
729
3
9
16
25
81
729
4
9
16
25
81
729
5
9
16
25
81
729
6
9
16
25
81
729
7
9
16
25
81
729
8
9
16
25
81
729
9
9
16
25
81
729
10
9
16
25
81
729
11
9
16
25
81
729
12
9
16
25
81
729
13
9
16
25
81
729
14
9
16
25
81
729
15
9
16
25
81
729
16
9
16
25
81
729
17
9
16
25
81
729
18
9
16
25
81
729
19
9
16
25
81
729
20
9
16
25
81
729
21
9
16
25
81
729
22
9
16
25
81
729
23
9
16
25
81
729
24
9
16
25
81
729
25
9
16
25
81
729
26
9
16
25
81
729
27
9
16
25
81
729
28
9
16
25
81
729
29
9
16
25
81
729
30
9
16
25
81
729
31
9
16
25
81
729
32
9
16
25
81
729
33
9
16
25
81
729
34
9
16
25
81
729
35
9
16
25
81
729
36
9
16
25
81
729
37
9
16
25
81
729
38
9
16
25
81
729
39
9
16
25
81
729
40
9
16
25
81
729
41
9
16
25
81
729
ARG1.2-R
90
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


/tmp/ipykernel_2774531/1378317126.py:10: RuntimeWarning: Mean of empty slice
  var = np.nanmean(var1)


25
81
729
29
9
16
25
81
729
30
9
16
25
81
729
31
9
16
25
81
729
32
9
16
25
81
729
33
9
16
25
81
729
34
9
16
25
81
729
35
9
16
25
81
729
36
9
16
25
81
729
37
9
16
25
81
729
38
9
16
25
81
729
39
9
16
25
81
729
40
9
16
25
81
729
41
9
16
25
81
729
42
9
16
25
81
729
43
9
16
25
81
729
44
9
16
25
81
729
45
9
16
25
81
729
46
9
16
25
81
729
47
9
16
25
81
729
48
9
16
25
81
729
49
9
16
25
81
729
50
9
16
25
81
729
51
9
16
25
81
729
52
9
16
25
81
729
53
9
16
25
81
729
54
9
16
25
81
729
55
9
16
25
81
729
56
9
16
25
81
729
57
9
16
25
81
729
58
9
16
25
81
729
59
9
16
25
81
729
60
9
16
25
81
729
61
9
16
25
81
729
62
9
16
25
81
729
63
9
16
25
81
729
64
9
16
25
81
729
65
9
16
25
81
729
66
9
16
25
81
729
67
9
16
25
81
729
68
9
16
25
81
729
69
9
16
25
81
729
70
9
16
25
81
729
71
9
16
25
81
729
72
9
16
25
81
729
73
9
16
25
81
729
74
9
16
25
81
729
75
9
16
25
81
729
76
9
16
25
81
729
77
9
16
25
81
729
78
9
16
25
81
729
79
9
16
25
81
729
80
9
16
25
81
729
81
9
16
25
81
729
82
9
16
25
81
729
83
9
16
25
81
729
